In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return files, targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test')

In [2]:
print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

There are 2750 total images.

There are 2000 training images.
There are 150 validation images.
There are 600 test images.


In [20]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [9]:
from keras.models import Model
from keras.applications.xception import preprocess_input, decode_predictions, Xception

xception = Xception(weights='imagenet')

xception_bottleneck = Model(inputs=xception.input, outputs=xception.get_layer('block14_sepconv2_act').output)

def Xception_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(xception.predict(img))

In [21]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')
valid_tensors = paths_to_tensor(valid_files).astype('float32')
test_tensors = paths_to_tensor(test_files).astype('float32')


 17%|█▋        | 335/2000 [00:44<05:16,  5.26it/s]


 33%|███▎      | 657/2000 [01:27<04:10,  5.37it/s]


 47%|████▋     | 946/2000 [02:08<02:01,  8.67it/s]


 63%|██████▎   | 1251/2000 [02:53<01:45,  7.12it/s]


 76%|███████▌  | 1524/2000 [03:38<01:27,  5.45it/s]


...

100%|██████████| 150/150 [00:37<00:00,  4.66it/s]

 16%|█▋        | 99/600 [00:37<03:15,  2.56it/s]


 44%|████▎     | 262/600 [01:40<03:05,  1.82it/s]


 70%|███████   | 422/600 [02:39<01:25,  2.08it/s]


 97%|█████████▋| 581/600 [03:46<00:07,  2.47it/s]


100%|██████████| 600/600 [03:53<00:00,  2.41it/s]


In [24]:
# Set the batch size higher if you can fit in in your GPU memory
batch_size = 250

datasets = {
    'train': train_tensors,
    'valid': valid_tensors,
    'test': test_tensors,    
}
bottleneck_features = {
    'train': None,
    'valid': None,
    'test': None,
}
for name, files in datasets.items():
    print(name)
    num_batches = len(files)//batch_size + 1
    for i in tqdm(range(num_batches)):
        s = i*batch_size
        e = (i+1)*batch_size
        input_ = preprocess_input(files[s:e])
        block14_features = xception_bottleneck.predict(input_)
        bottleneck_features[name] = np.concatenate((bottleneck_features[name], block14_features)) if bottleneck_features[name] is not None else block14_features

np.savez('bottleneck_features.npz', **bottleneck_features)


  0%|          | 0/3 [00:00<?, ?it/s]

test



100%|██████████| 3/3 [00:17<00:00,  5.90s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

valid



100%|██████████| 1/1 [00:04<00:00,  4.38s/it]

  0%|          | 0/9 [00:00<?, ?it/s]

train



 89%|████████▉ | 8/9 [00:58<00:07,  7.47s/it]

ValueError: all the input arrays must have same number of dimensions

In [ ]:
bottleneck_features = np.load('bottleneck_features.npz')
train_features = bottleneck_features['train']
valid_features = bottleneck_features['valid']
test_features = bottleneck_features['test']

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()

model.add(Dense(100, activation='relu'))
model.add(Dense(3), activation='softmax')

model.summary()


In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 10

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1, shuffle=True)

In [ ]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)